### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [129]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [45]:
total_players = len(purchase_data["SN"].unique())
df_total_players = pd.DataFrame({"Total Players": [total_players]})
df_total_players

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# Calculate Purchase Analysis metrics
num_items = len(purchase_data["Item ID"].unique())
avg_price = np.round(purchase_data["Price"].mean(),decimals=2)
num_purchase = len(purchase_data)
total_revenue = np.round(purchase_data["Price"].sum(), decimals=2)

# Create summary dataframe
df_summary = pd.DataFrame({"Number of Unique Items": [num_items], "Average Price": avg_price,
                           "Number of Purchases": num_purchase, "Total Revenue": total_revenue})

# come back and reformat

df_summary.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.05,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# really nifty - sort by "Gender" but filter by unique entries in "SN"
# link -- https://stackoverflow.com/questions/38309729/count-unique-values-with-pandas-per-groups

# still have to reformat "Percentage of Players" column with percentage sign
gb = purchase_data.groupby(["Gender"])["SN"].nunique()
total_count = [x for x in gb]
percentage_of_players = [np.round(x/np.sum(total_count),decimals=4)*100 for x in gb]

df_gender = pd.DataFrame({"Total Count":total_count, "Percentage of Players": percentage_of_players},
                        index=["Female","Male","Other / Non-Disclosed"])
df_gender

,Total Count,Percentage of Players
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
gb2 = purchase_data.groupby(["Gender"])
gb2 = dict(list(gb2))
gb2["Female"]["Price"].mean()

3.203008849557519

In [6]:
# purchase count
purchase_count = [len(gb2["Female"]), len(gb2["Male"]), len(gb2["Other / Non-Disclosed"])]

# average purchase price
avg_purchase_price = [gb2["Female"]["Price"].mean(), gb2["Male"]["Price"].mean(), gb2["Other / Non-Disclosed"]["Price"].mean()]
avg_purchase_price = [np.round(x,decimals=2) for x in avg_purchase_price]

# total purchase value
total_purchase_value = [gb2["Female"]["Price"].sum(), gb2["Male"]["Price"].sum(), gb2["Other / Non-Disclosed"]["Price"].sum()]

In [7]:
# Create function to calculate amount of money spent per gender
def dict_totals_per_gender(gender_key):
    """ Docstring: dict_totals_per_gender takes input key (as a string) and returns a dictionary of all unique
    players for that gender with their corresonding purchase total. Also returns the mean value spent per gender.
    To be used for general gender purchase analysis"""
    
    # grab list of unique names
    unique_names = list(gb2[gender_key]["SN"].unique())
    # preallocate a dictionary with keys as screen name and values as zero
    totals = {keys:0 for keys in unique_names}
    
    # for every key in totals, pull a subset dataframe consisting of only that person's purchases and sum them
    for key in totals:
        df_temp = purchase_data.loc[purchase_data["SN"]==key,:]
        totals[key] += df_temp["Price"].sum()
    
    return totals, pd.Series([totals[k] for k in totals]).mean()

## Use function declared above to calculate spend per gender ##
# Avg Total Per Person
f_total, f_mean = dict_totals_per_gender("Female")
m_total, m_mean = dict_totals_per_gender("Male")
o_total, o_mean = dict_totals_per_gender("Other / Non-Disclosed")

avg_price_per_gender = [f_mean, m_mean, o_mean]
avg_price_per_gender = list(np.round(np.asarray(avg_price_per_gender),decimals=2))   

In [8]:
## Construct dataframe ## - come back to reformat
df_gender_purchasing = pd.DataFrame({"Purchase Count": purchase_count, "Average Purchase Price": avg_purchase_price,
                                     "Total Purchase Value": total_purchase_value, "Average Total Per Person": avg_price_per_gender},
                                   index=["Female","Male","Other / Non-Disclosed"])
df_gender_purchasing

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Per Person
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [21]:
## Come back: need to reformat and check values in Total count ##
# Check with TA's - looks like charts in Age Demographics and Purchase Analysis are different (even though
# though they should be the same)

# create bins for cut
bins = [0,10,15,20,25,30,35,40,purchase_data["Age"].max() + 1]
#bins = [0,9,14,19,24,29,34,39,purchase_data["Age"].max() + 1]
# create bin labels
bin_labels = ["<10", "10-14","15-19","20-24","25-29","30-34","35-39","40+"]
# preallocate dataframe
df_age_demographics = pd.DataFrame()
# pull the age column from purchase data
df_age_demographics["Age"] = purchase_data["Age"]
# cut the purchase_data DataFrame to categorize population by age group
df_age_demographics["Age Groups"] = pd.cut(purchase_data["Age"],bins=bins, labels=bin_labels, include_lowest=True, right=False)
# group by age categories we defined in bin labels
gb3 = df_age_demographics.groupby("Age Groups").count()
# format percentage of age group
percent_age_group = [age/gb3["Age"].sum()*100 for age in gb3["Age"]]
percent_age_group = np.round(percent_age_group,decimals=2)

# build summary DataFrame
df_age_demographics = pd.DataFrame({"Total Count": gb3["Age"], "Percentage of Players": percent_age_group},
                                   index=bin_labels)

# display dataframe
df_age_demographics

,Total Count,Percentage of Players
<10,23,2.95
10-14,28,3.59
15-19,136,17.44
20-24,365,46.79
25-29,101,12.95
30-34,73,9.36
35-39,41,5.26
40+,13,1.67


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [47]:
# come back and reformat
# Create function to calculate amount of money spent per age bracket
def dict_totals_per_age(df_temp):
    """ Docstring: dict_totals_per_age takes input key (as a string) and returns a dictionary of all unique
    players for that age group with their corresonding purchase total. Also returns the mean value spent per age bracket.
    To be used for general age purchase analysis"""
    
    # grab list of unique names
    unique_names = list(df_temp["SN"].unique())
    # preallocate a dictionary with keys as screen name and values as zero
    totals = {keys:0 for keys in unique_names}
    
    # for every key in totals, pull a subset dataframe consisting of only that person's purchases and sum them
    for key in totals:
        df_temp = purchase_data.loc[purchase_data["SN"]==key,:]
        totals[key] += df_temp["Price"].sum()
        
    avg_price = pd.Series([totals[k] for k in totals]).mean()
    
    return totals, avg_price

In [93]:
list_avg_total_per_age = []
avg_purchase_price = []
total_purchase_price = []

# under 10
df_under10 = purchase_data.loc[purchase_data["Age"] < 10,:]
totals, avg_price = dict_totals_per_age(df_under10)
avg_purchase_price.append(df_under10["Price"].mean())
total_purchase_price.append(df_under10["Price"].sum())
list_avg_total_per_age.append(avg_price)

# 10 to 14
df_10to14 = purchase_data.loc[(purchase_data["Age"] >= 10) & (purchase_data["Age"] < 15),:]
totals, avg_price = dict_totals_per_age(df_10to14)
avg_purchase_price.append(df_10to14["Price"].mean())
total_purchase_price.append(df_10to14["Price"].sum())
list_avg_total_per_age.append(avg_price)

# 15 to 19
df_15to19 = purchase_data.loc[(purchase_data["Age"] >= 15) & (purchase_data["Age"] < 20),:]
totals, avg_price = dict_totals_per_age(df_15to19)
avg_purchase_price.append(df_15to19["Price"].mean())
total_purchase_price.append(df_15to19["Price"].sum())
list_avg_total_per_age.append(avg_price)

# 20 to 24
df_20to24 = purchase_data.loc[(purchase_data["Age"] >=20) & (purchase_data["Age"] < 25), :]
totals, avg_price = dict_totals_per_age(df_20to24)
avg_purchase_price.append(df_20to24["Price"].mean())
total_purchase_price.append(df_20to24["Price"].sum())
list_avg_total_per_age.append(avg_price)

# 25 to 29
df_25to29 = purchase_data.loc[(purchase_data["Age"] >=25) & (purchase_data["Age"] < 30), :]
totals, avg_price = dict_totals_per_age(df_25to29)
avg_purchase_price.append(df_25to29["Price"].mean())
total_purchase_price.append(df_25to29["Price"].sum())
list_avg_total_per_age.append(avg_price)

# 30 to 34
df_30to34 = purchase_data.loc[(purchase_data["Age"] >=30) & (purchase_data["Age"] < 35), :]
totals, avg_price = dict_totals_per_age(df_30to34)
avg_purchase_price.append(df_30to34["Price"].mean())
total_purchase_price.append(df_30to34["Price"].sum())
list_avg_total_per_age.append(avg_price)

# 35 to 39
df_35to39 = purchase_data.loc[(purchase_data["Age"] >=35) & (purchase_data["Age"] < 40), :]
totals, avg_price = dict_totals_per_age(df_35to39)
avg_purchase_price.append(df_35to39["Price"].mean())
total_purchase_price.append(df_35to39["Price"].sum())
list_avg_total_per_age.append(avg_price)

df_over40 = purchase_data.loc[purchase_data["Age"] >= 40, :]
totals, avg_price = dict_totals_per_age(df_over40)
avg_purchase_price.append(df_over40["Price"].mean())
total_purchase_price.append(df_over40["Price"].sum())
list_avg_total_per_age.append(avg_price)

In [98]:
# round all averages to two decimals
list_avg_total_per_age = [np.round(x, decimals=2) for x in list_avg_total_per_age]
avg_purchase_price = [np.round(x, decimals=2) for x in avg_purchase_price]
total_purchase_price = [np.round(x, decimals=2) for x in total_purchase_price]

In [105]:
df_purchase_analysis = pd.DataFrame({"Purchase Count": gb3["Age"], "Average Purchase Price": avg_purchase_price,
                                    "Total Purchase Price": total_purchase_price, 
                                    "Avg Total Price Per Person": list_avg_total_per_age})
df_purchase_analysis

,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Price Per Person
Age Groups,,,,
<10,23,3.35,77.13,4.54
10-14,28,2.96,82.78,3.76
15-19,136,3.04,412.89,3.86
20-24,365,3.05,1114.06,4.32
25-29,101,2.90,293.00,3.81
30-34,73,2.93,214.00,4.12
35-39,41,3.60,147.67,4.76
40+,13,2.94,38.24,3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [154]:
# iterate through two dictionaries to store keys (screen name) and values (either number of purchases or total spent)
# can use this later to create the data frame

# grab list of unique names
unique_names = list(purchase_data["SN"].unique())

# preallocate a dictionary with keys as screen name and values as zero
purchase_count = {keys:0 for keys in unique_names}
total = {keys:0 for keys in unique_names}

# for every key in totals, pull a subset dataframe consisting of only that person's purchases and sum them
for ii in range(0,len(purchase_data)):
    # get screen name from row ii
    screen_name = purchase_data["SN"][ii]
    
    # number of purchases
    purchase_count[screen_name] += 1
    
    # total
    total[screen_name] += purchase_data["Price"][ii]
    

In [156]:
# construct data frame with dictionaries previously constructed
df_top_spenders = pd.DataFrame({"Purchase Count": purchase_count,"Total Purchase Value": total})
# calculate average purchase price column for dataframe
df_top_spenders["Average Purchase Price"] = np.round(df_top_spenders["Total Purchase Value"] / df_top_spenders["Purchase Count"] , decimals=2) 
# reorder columns
df_top_spenders = df_top_spenders[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
# sort Data Frame and print top 5 spenders
df_top_spenders.sort_values(by="Total Purchase Value",ascending=False).head(5)



,Purchase Count,Average Purchase Price,Total Purchase Value
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

